In [1]:
cd ../app

/home/byl/Downloads/Program/Parking/app


In [2]:
from lib.data import ParkingData
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO, DQN

DATA_DIR = 'data'

data_obj = ParkingData(data_dir=DATA_DIR)

23/01/10 19:31:45 WARN Utils: Your hostname, byl-i15-7000 resolves to a loopback address: 127.0.1.1; using 192.168.1.239 instead (on interface wlp3s0)
23/01/10 19:31:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/10 19:31:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/10 19:31:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/10 19:31:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
data_obj.write_ml()
data_obj.load_ml()

train {'start': '2020-01-01', 'end': '2020-04-30'}


23/01/10 19:32:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


test {'start': '2020-05-01', 'end': '2020-05-31'}
23/01/10 19:32:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [8]:
import numpy as np
import gym
from gym import spaces


TRUE_POSITIVE_SCORE = 10
TRUE_NEGATIVE_SCORE = 1


class ParkingEnv(gym.Env):
    def __init__(
        self,
        data_obj,
        id_col,
        y_col,
        x_cols,
        x_range,
        emb_col,
        emb_range,
        sample_size=0.01,
        key='train',
    ):
        super(ParkingEnv, self).__init__()

        self.data_obj = data_obj
        self.id_col = id_col
        self.y_col = y_col
        self.x_cols = x_cols
        self.x_range = x_range
        self.emb_col = emb_col
        self.emb_range = emb_range
        self.obs_size = len(x_cols) + data_obj.emb_size
        self.sample_size = sample_size
        self.key = key

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(low=np.array([x_range[0] for i in x_cols] + [emb_range[0]] * data_obj.emb_size), 
                                            high=np.array([x_range[1] for i in x_cols] + [emb_range[1]] * data_obj.emb_size), 
                                            shape=(self.obs_size,), 
                                            dtype=np.float64)
        self.load_batch()
        self.reset()
    
    def load_batch(self):
        self.batch = self.data_obj.sample(key=self.key, sample_size=self.sample_size)

    def step(self, action):
        if action == self.__hist__():
            if action == 1:
                reward = TRUE_POSITIVE_SCORE
            else:
                reward = TRUE_NEGATIVE_SCORE
        else:
            reward = 0
        return self.__obs__(), reward, True, {}

    def reset(self):
        try:
            self.current = self.batch.pop()
        except IndexError:
            self.load_batch()
            self.current = self.batch.pop()
        return self.__obs__()

    def __obs__(self):
        return np.array([self.current[col] for col in self.x_cols] + self.current[self.emb_col])

    def __hist__(self):
        return self.current[self.y_col]


In [9]:
id_col = 'BayId'
y_col = 'InViolation'
x_cols = [
    'is_holiday',
    'WeekDay',
    'no_br_record',
    'EffectiveOnPH',
    'LogDuration',
    'LogDisabilityExt',
    'loading_zone',
    'disabled',
    'meter',
    'clearway',
    'no_stopping',
    'no_parking',
    'ticket',
    'letter_p',
    'exempt_other',
    'exempt_resident',
    'exempt_disable',
    'hr_from_start',
    'hr_to_end',
    'hr',
    'VehiclePresent'
]
x_range = [0, 1]
emb_col = 'emb'
emb_range = [-7, 7]

env = ParkingEnv(
    data_obj=data_obj,
    id_col=id_col,
    y_col=y_col,
    x_cols=x_cols,
    x_range=x_range,
    emb_col=emb_col,
    emb_range=emb_range,
    key='train'
)

/home/byl/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(


In [10]:
check_env(env)

In [12]:
model = PPO(
    "MlpPolicy", 
    env, 
    verbose=0,
    tensorboard_log='tmp/tensorboard/ppo'
)
model.learn(total_timesteps=10_000)

In [ ]:
# model = DQN(
#     "MlpPolicy", 
#     env, 
#     verbose=0,
#     tensorboard_log='tmp/tensorboard/dqn'
# )
# model.learn(total_timesteps=2_000_000)

In [ ]:
# model.save('model_10_1_2M')

In [13]:
eval_env = ParkingEnv(
    data_obj=data_obj,
    id_col=id_col,
    y_col=y_col,
    x_cols=x_cols,
    x_range=x_range,
    emb_col=emb_col,
    emb_range=emb_range,
    key='test'
)

/home/byl/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(


In [14]:
rewards = []
actions = []
for i in range(10_000):
    obs = eval_env.reset()
    action = model.predict(obs, deterministic=True)[0]
    obs, reward, done, info = eval_env.step(action)
    if done == True:
        obs = eval_env.reset()
    
    rewards.append(reward)
    actions.append(action)
rewards = np.array(rewards)
actions = np.array(actions)
print(np.mean(rewards))
print(np.mean(actions))

1.0099
0.0438


In [15]:
print('action = 1, reward = 1,', (rewards[actions==1]>0).sum())
print('action = 1, reward = 0,', (actions==1).sum() - (rewards[actions==1]>0).sum())
print('action = 0, reward = 1,', (rewards[actions==0]>0).sum())
print('action = 0, reward = 0,', (actions==0).sum() - (rewards[actions==0]>0).sum())

action = 1, reward = 1, 115
action = 1, reward = 0, 323
action = 0, reward = 1, 8949
action = 0, reward = 0, 613
